In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

In [2]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

# 卷积神经网络（LeNet）

**LeNet（LeNet-5）由两个部分组成： 卷积编码器和全连接层密集块**

In [3]:
// class Reshape(torch.nn.Module):
//     def forward(self, x):
//         return x.view(-1, 1, 28, 28)

// net = torch.nn.Sequential(Reshape(), nn.Conv2d(1, 6, kernel_size=5,
//                                                padding=2), nn.Sigmoid(),
//                           nn.AvgPool2d(kernel_size=2, stride=2),
//                           nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
//                           nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
//                           nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
//                           nn.Linear(120, 84), nn.Sigmoid(), nn.Linear(84, 10))

In [4]:
struct Reshape : torch::nn::Module {
  torch::Tensor forward(torch::Tensor X) {
    return X.view({-1, 1, 28, 28});
  }
};

In [5]:
torch::nn::Sequential net(Reshape(),
                          torch::nn::Conv2d(torch::nn::Conv2dOptions(1, 6, 5).padding(2).bias(false)),
                          torch::nn::Sigmoid(),
                          torch::nn::AvgPool2d(torch::nn::AvgPool2dOptions(2).stride(2)),
                          torch::nn::Conv2d(torch::nn::Conv2dOptions(6, 16, 5).bias(false)),
                          torch::nn::Sigmoid(),
                          torch::nn::AvgPool2d(torch::nn::AvgPool2dOptions(2).stride(2)),
                          torch::nn::Flatten(),
                          torch::nn::Linear(16 * 5 * 5, 120),
                          torch::nn::Sigmoid(),
                          torch::nn::Linear(120, 84),
                          torch::nn::Sigmoid(),
                          torch::nn::Linear(84, 10)
                         );

**检查模型**

In [6]:
// X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
// for layer in net:
//     X = layer(X)
//     print(layer.__class__.__name__, 'output shape: \t', X.shape)

In [7]:
auto X = torch::rand({1, 1, 28, 28}, torch::kFloat);

auto& model_ref = *net; 

for (auto module : model_ref)
{
    std::cout << module.ptr()->name() << std::endl;
    X = module.forward(X);
    std::cout << "output shape: \t" << X.sizes() << std::endl;
//     std::cout << module.ptr()->parameters() << std::endl;
}

__cling_N53::Reshape
output shape: 	[1, 1, 28, 28]
torch::nn::Conv2dImpl
output shape: 	[1, 6, 28, 28]
torch::nn::SigmoidImpl
output shape: 	[1, 6, 28, 28]
torch::nn::AvgPool2dImpl
output shape: 	[1, 6, 14, 14]
torch::nn::Conv2dImpl
output shape: 	[1, 16, 10, 10]
torch::nn::SigmoidImpl
output shape: 	[1, 16, 10, 10]
torch::nn::AvgPool2dImpl
output shape: 	[1, 16, 5, 5]
torch::nn::FlattenImpl
output shape: 	[1, 400]
torch::nn::LinearImpl
output shape: 	[1, 120]
torch::nn::SigmoidImpl
output shape: 	[1, 120]
torch::nn::LinearImpl
output shape: 	[1, 84]
torch::nn::SigmoidImpl
output shape: 	[1, 84]
torch::nn::LinearImpl
output shape: 	[1, 10]


**LeNet在Fashion-MNIST数据集上的表现**